<a href="https://colab.research.google.com/github/VarjuAkos/Onlab/blob/main/LeagalEasy_Fine_tuning_a_Model_to_Produce_Natural_Language_Summaries_of_Legalese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LegalEasy: Fine-tuning a Model to Produce Natural Language Descriptions of Legalese

Today we'll be looking at a specific example of fine-tuning [Mistral-7B-Instruct-v0.2](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2): Summarization in a particular style for a particular domain!

Let's start, as we always do, with installing our dependencies.



In [ ]:
!pip install -qU transformers peft trl accelerate bitsandbytes datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00


Next, let's set-up some data!

## Data

We'll be using the Legal Summarization [dataset](https://github.com/lauramanor/legal_summarization) from the paper [Plain English Summarization of Contracts](https://www.aclweb.org/anthology/W19-2201) today.

This dataset contains pairs in the following format:

- Original Text: A blob of legal text, think ToS
- Reference Summary: A short natural language summary of the legal text

We'll start by cloning the repository containing our data.

In [ ]:
!git clone https://github.com/lauramanor/legal_summarization.git

Cloning into 'legal_summarization'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 31 (delta 2), reused 0 (delta 0), pack-reused 25
Receiving objects: 100% (31/31), 136.60 KiB | 19.51 MiB/s, done.
Resolving deltas: 100% (10/10), done.


Let's convert this into an expected format - in this case a list of `json` objects.

In [ ]:
import json

jsonl_array = []

with open('legal_summarization/tldrlegal_v1.json') as f:
  data = json.load(f)
  for key, value in data.items():
    jsonl_array.append(value)

Now we can convert that into the desired format for our fine-tuning - a Hugging Face `Dataset`!

In [ ]:
from datasets import Dataset, load_dataset

legal_dataset = Dataset.from_list(jsonl_array)

Let's see how many items we're working with in our dataset.

> NOTE: Keep in mind that this is a relatively simple example meant to showcase fine-tuning - in practice, we'd want to use somewhere in the neighbourhood of ~500-50,000 examples.

In [ ]:
legal_dataset

Dataset({
    features: ['doc', 'id', 'original_text', 'reference_summary', 'title', 'uid'],
    num_rows: 85
})

Let's look at an example of our original text and summary!

In [ ]:
print(f"Original Text: {legal_dataset[0]['original_text']}\n\nSummary: {legal_dataset[0]['reference_summary']}")

Original Text: welcome to the pokémon go video game services which are accessible via the niantic inc niantic mobile device application the app. to make these pokémon go terms of service the terms easier to read our video game services the app and our websites located at http pokemongo nianticlabs com and http www pokemongolive com the site are collectively called the services. please read carefully these terms our trainer guidelines and our privacy policy because they govern your use of our services.

Summary: hi.


Now, we mentioned earlier we were going to fine-tune Mistral-7B-Instruct-v0.2, which is important for our next step: Creating the instruction format.

Let's take a look at the instructions (so meta) to generate an instruction prompt from the [model card](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2#instruction-format)


> In order to leverage instruction fine-tuning, your prompt should be surrounded by [INST] and [/INST] tokens. The very first instruction should begin with a begin of sentence id. The next instructions should not. The assistant generation will be ended by the end-of-sentence token id.

Let's look at an example of how we might format our instruction - and then reproduce that in code.

```python
<s>[INST]Please convert the following legal content into a human-readable summary

[LEGAL_DOC]
welcome to the pokémon go video game services which are accessible via the niantic inc niantic mobile device application the app. to make these pokémon go terms of service the terms easier to read our video game services the app and our websites located at http pokemongo nianticlabs com and http www pokemongolive com the site are collectively called the services. please read carefully these terms our trainer guidelines and our privacy policy because they govern your use of our services.
[END_LEGAL_DOC]

[/INST]
hi.</s>
```

> NOTE: We're adding our own special tokens here in `[LEGAL_DOC]` and `[END_LEGAL_DOC]` to encourage the model to better understand our context, but these are not special tokens that are already understood by the model

In [ ]:
INSTRUCTION_PROMPT_TEMPLATE = """\
<s>[INST]Please convert the following legal content into a human-readable summary

[LEGAL_DOC]
{LEGAL_TEXT}
[END_LEGAL_DOC]

[/INST]
"""

RESPONSE_TEMPLATE = """\
{NATURAL_LANGUAGE_SUMMARY}</s>"""

Now we can create a helper function that will convert our dataset row into the above prompt!

In [ ]:
def create_instruction(sample, return_response=True):
  prompt = INSTRUCTION_PROMPT_TEMPLATE.format(LEGAL_TEXT=sample["original_text"])

  if return_response:
    prompt += RESPONSE_TEMPLATE.format(NATURAL_LANGUAGE_SUMMARY=sample["reference_summary"])

  return prompt

Let's try it out!

In [ ]:
create_instruction(legal_dataset[0])

'<s>[INST]Please convert the following legal content into a human-readable summary\n\n[LEGAL_DOC]\nwelcome to the pokémon go video game services which are accessible via the niantic inc niantic mobile device application the app. to make these pokémon go terms of service the terms easier to read our video game services the app and our websites located at http pokemongo nianticlabs com and http www pokemongolive com the site are collectively called the services. please read carefully these terms our trainer guidelines and our privacy policy because they govern your use of our services.\n[END_LEGAL_DOC]\n\n[/INST]\nhi.</s>'

We'll partition our dataset so we can test some of the outputs after we've completed our training.

In [ ]:
prepared_legal_dataset = legal_dataset.train_test_split(test_size=0.1)

In [ ]:
prepared_legal_dataset

DatasetDict({
    train: Dataset({
        features: ['doc', 'id', 'original_text', 'reference_summary', 'title', 'uid'],
        num_rows: 76
    })
    test: Dataset({
        features: ['doc', 'id', 'original_text', 'reference_summary', 'title', 'uid'],
        num_rows: 9
    })
})

## Loading Our Model

Now we can move onto loading our model!

We're going to be dependent on two major technologies to allow us to train our model with <=16GB GPU RAM.

1. Quantization
2. LoRA

> NOTE: We've done some events on [LoRA](https://www.youtube.com/watch?v=kV8yXIUC5_4&list=PLrSHiQgy4VjGMzyXsSlvN-TjPaqFFsAGP&index=4) and [QLoRA](https://www.youtube.com/watch?v=XOb-djcw6hs&list=PLrSHiQgy4VjGMzyXsSlvN-TjPaqFFsAGP&index=5) for deeper dives into those respective technologies

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

We'll load our tokenizer and do a few pre-processing steps to prepare it for training.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

Now we can set-up our LoRA configuration file - which will let the TRL library know how to create our LoRA adapters!

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM",
)

### Fine-tuning!

Now onto the star of today's show: Fine-tuning!

We're going to use the `SFTTrainer` or "Supervised Fine-tuning Trainer" from the [TRL](https://github.com/huggingface/trl/tree/main) library today.

In essence, this is a trainer that will handle most of the fine-tuning process for us - including but not limited to:

- Dataset packing
- LoRA initialization
- Tokenizing

Let's set up some training hyper-parameters through transformers `TrainingArguments` class to get started. Here's a breakdown of which parameters are doing what:

- `outpur_dir` - indicates where we store the results of training locally
- `num_train_epochs` - how many epochs we will train for (somewhere ~3-4 is a good place to start)
- `per_device_train_batch_size` - how many batches we want per device. In this case, we only have one device - but we set this to a low value to keep memory consumption below 16GB GPU RAM
- `gradient_accumulation_steps` - this hyper-parameter lets us indicate how many steps we wish to accumulate our gradients for, this is a way to "cheat out" a larger batch size without extra memory consumption
- `gradient_checkpointing` - this lets us [trade off memory consumption for increased training time](https://huggingface.co/docs/transformers/v4.18.0/en/performance#gradient-checkpointing)
- `optim` - our optimizer! In this case, we're using  a fused ADAMW optimiser. The fused method is a faster version of the ADAMW optimizer but is reliant on CUDA (GPU). More information can be read [here](https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html)

The rest of the hyper-parameters are taken directly from the QLoRA [paper](https://arxiv.org/abs/2305.14314) and are discussed in more detail there!

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="leagaleasy-mistral-7b-instruct-v0.2-v1",
    num_train_epochs=4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    push_to_hub=True,
)

Because we're going to automatically push our model to the hub, thanks to `push_to_hub=True`, we'll want to provide a Hugging Face Write token.

> NOTE: You can skip this step by commenting out `push_to_hub=True`

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Now, finally, we can set-up our `SFTTrainer` which is going to help us fine-tune this model on our dataset we create at the beginning of the notebook!

We'll discuss a few parameters to clarify what they're doing:

- `formatting_func` - since we created a helper function to convert our dataset row into a Mistral-style Instruction prompt, we need to let TRL know to use it to create our prompts!
- `peft_config` - TRL will automatically load our model in LoRA format with this config.
- `packing` - this will let our model "pack" the context window to ensure we're not wasting precious memory on padding tokens where posssible
- `dataset_kwargs` - because we already added the special tokens to our prompts, we want to ensure we don't "re-add" them!

With those parameters set - we're clear for training!

In [ ]:
from trl import SFTTrainer

max_seq_length=2048

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=prepared_legal_dataset["train"],
    formatting_func=create_instruction,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    dataset_kwargs={
        "add_special_tokens" : False,
        "append_concat_token" : False,
    }
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:434: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictionary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

All that's left to do is fine-tune our model!

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


Checkpoint destination directory de-legalese-mistral-7b-instruct-v0.2-v1/checkpoint-3 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Checkpoint destination directory de-legalese-mistral-7b-instruct-v0.2-v1/checkpoint-6 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of u

TrainOutput(global_step=12, training_loss=0.8033531506856283, metrics={'train_runtime': 84.1045, 'train_samples_per_second': 0.571, 'train_steps_per_second': 0.143, 'total_flos': 4243516003713024.0, 'train_loss': 0.8033531506856283, 'epoch': 4.0})

Now we can save it.

In [ ]:
trainer.save_model()

events.out.tfevents.1709742672.282a8b38838c.6548.0:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

Let's clear up memory so we can do inference while staying under our memory budget.

In [ ]:
del model
del trainer
torch.cuda.empty_cache()

We'll need to load our mode back as a PEFT model, due to the use of LoRA, and then merge the LoRA layers back into the original model for use in Hugging Face pipelines.

In [ ]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto"
)

merged_model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Now we can load our pipeline for `text-generation`.

In [ ]:
from transformers import pipeline

text_pipe = pipeline("text-generation", merged_model, tokenizer=tokenizer)

## Testing Fine-tuned Model

Now that we've fine-tuned, lets see how we did!

In [ ]:
prepared_legal_dataset["test"][0]["original_text"]

'one major rulethe one major rule is that you must not distribute anything we ve made. by distribute anything we ve made what we mean is give copies of the game away make commercial use of try to make money from or let other people get access to our game and its parts in a way that is unfair or unreasonable. so the one major rule is that unless we specifically agree it such as in brand and asset usage guidelines you must not give copies of our game to anyone else make commercial use of anything we ve made try to make money from anything we ve made orlet other people get access to anything we ve made in a way that is unfair or unreasonable and so that we are crystal clear what we have made includes but is not limited to the client or the server software for our game. it also includes modified versions of a game part of it or anything else we ve made. otherwise we are quite relaxed about what you do in fact we really encourage you to do cool stuff but just don t do those things that we s

In [ ]:
outputs = text_pipe(create_instruction(prepared_legal_dataset["test"][0], return_response=False), max_new_tokens=256, temperature=0.1, top_k=50, top_p=0.1, return_full_text=False)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
outputs

[{'generated_text': 'don t distribute our game.'}]

In [ ]:
prepared_legal_dataset["test"][0]["reference_summary"]

'you may not give copies of the game to anyone else or try to make money from anything we ve made.'

Another example!

In [ ]:
prepared_legal_dataset["test"][1]["original_text"]

'you agree to use the sdk and write applications only for purposes that are permitted by a the license agreement and b any applicable law regulation or generally accepted practices or guidelines in the relevant jurisdictions including any laws regarding the export of data or software to and from the united states or other relevant countries.'

In [ ]:
outputs = text_pipe(create_instruction(prepared_legal_dataset["test"][1], return_response=False), max_new_tokens=256, temperature=0.1, top_k=50, top_p=0.1, return_full_text=False)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
outputs

[{'generated_text': 'don t use the sdk for illegal stuff.'}]

In [ ]:
prepared_legal_dataset["test"][1]["reference_summary"]

'stay within the law and license agreement.'

Overall, our fine-tuning did a great job and allowed our model to generate our desired output - all with <16GB GPU memory, and 4 epochs of fine-tuning!